# Assignment 3.3: Automated ETL Pipeline

####Geovanny Peña Rueda
**Purpose:**  
Automate the ETL pipeline to clean and label STEDI Step Test sensor data. This notebook:
- Loads raw device messages and step tests
- Converts and cleans columns
- Labels sensor readings as `step` or `no_step`
- Saves the curated dataset to the Silver layer
- Performs verification queries

## Part 1 – Load Raw Bronze Tables


In [0]:
df_device = spark.table("workspace.bronze.device_messages_raw")
df_steps = spark.table("workspace.bronze.rapid_step_tests_raw")


## Part 2 – Prepare Each Table


In [0]:
from pyspark.sql.functions import regexp_extract, col, lit, when

# Convert 'distance' from string to integer (distance_cm)
df_device = df_device.withColumn(
    "distance_cm",
    regexp_extract(col("distance"), r"(\d+)", 1).cast("int")
)

# Add source labels for traceability
df_device = df_device.withColumn("source", lit("device"))
df_steps = df_steps.withColumn("source", lit("step"))


## Part 3 – Label Each Sensor Reading (step / no_step)


In [0]:
# Extract the step test window (start and stop times)
df_steps_window = df_steps.select("device_id", "start_time", "stop_time")

# Label each sensor reading as 'step' or 'no_step'
df_labeled = (
    df_device.alias("d")
    .join(
        df_steps_window.alias("s"),
        (col("d.device_id") == col("s.device_id")) &
        (col("d.timestamp").between(col("s.start_time"), col("s.stop_time"))),
        "left"
    )
    .withColumn(
        "step_label",
        when(col("s.start_time").isNotNull(), "step").otherwise("no_step")
    )
)



## Part 4 – Select Final Curated Columns


In [0]:
from pyspark.sql.functions import col

df_final = df_labeled.select(
    col("d.timestamp").alias("timestamp"),
    col("d.sensor_type").alias("sensor_type"),
    col("distance_cm"),
    col("d.device_id").alias("device_id"),
    col("step_label"),
    col("source")
)

display(df_final)


## Part 5 – Save Curated Silver Dataset

In [0]:
spark.sql("USE workspace.silver")

# Create a temporary view so SQL can see it
df_final.createOrReplaceTempView("df_final_temp")

In [0]:
%sql
-- This is the required final SQL cell for the ETL Job

CREATE OR REPLACE TABLE labeled_step_test AS
SELECT * FROM df_final_temp;



## Part 6 – Verification Queries


In [0]:
%sql
-- 1. Steps vs No-Steps
SELECT step_label, COUNT(*) AS row_count
FROM labeled_step_test
GROUP BY step_label;

-- 2. Invalid or missing step labels
SELECT *
FROM labeled_step_test
WHERE step_label NOT IN ('step', 'no_step')
OR step_label IS NULL
LIMIT 50;

-- 3. Source label counts
SELECT source, COUNT(*) AS row_count
FROM labeled_step_test
GROUP BY source;


-- 4. Invalid or missing source labels
SELECT *
FROM labeled_step_test
WHERE source NOT IN ('device','step')
OR source IS NULL
LIMIT 50;


## Part 7 – Ethics Reflection

When automating health-related data pipelines, engineers must consider ethical responsibilities including:

- **Privacy and Security:** Ensure personal and health data is protected.
- **Data Accuracy and Validation:** Avoid labeling errors that could mislead analyses.
- **Avoiding Bias:** Ensure models and labels do not unfairly target certain groups.
- **Not Making Medical Claims:** Clearly communicate that the data is for analysis, not diagnosis.
- **Protecting People, Not Just Data:** Ensure decisions from automated pipelines do not harm users.

Automation must be done carefully to build trustworthy and ethical health data systems.
